# Efficient quantum simulation using Tensor Network states

The aim of this notebook is to demonstrate the impact of the Tensor Network state representation
on computational complexity in quantum computing simulations. During the morning lecture, the asymptotic complexity of
TN methods was discussed in an abstract manner, and now we will see, through some concrete examples via the ITensors package
precisely what the practical difference in runtimes is.

In [1]:
using LinearAlgebra
using ITensors

# Exercise 1: 5 minutes 

Complete the code in the following function to compute the Kronecker product ⊗ of vectors/matrices a,b.
Note that the Kronecker product gives the abstract tensor product expressed in a particular basis.
https://en.wikipedia.org/wiki/Kronecker_product

In [2]:
function tensor(a,b)
    return nothing #fill this in so the test cell below passes
end

tensor (generic function with 1 method)

run the below cell to test your implementation of tensor

In [3]:
function test_tensor()
    @assert tensor([1 2; 3 4], [5 6; 7 8]) == [5 6 10 12; 7 8 14 16; 15 18 20 24; 21 24 28 32]
    @assert tensor([1, 2], [3, 4]) == [3, 4, 6, 8]
    return nothing
end

test_tensor()

LoadError: AssertionError: tensor([1 2; 3 4], [5 6; 7 8]) == [5 6 10 12; 7 8 14 16; 15 18 20 24; 21 24 28 32]

hint: there is an inbuilt kron function in the language

solution:

function tensor(a,b)
    return kron(a,b)
end

## Key takeaway: taking the tensor product multiplies all the dimensions of the inputs

# Exercise 2: 5 minutes

Using the represenation |0> = [1, 0] and |1> = [0, 1], compute for n qubits,
GHZ state:      (|0> ⊗ ... ⊗ |0>) + (|1> ⊗ ... |1>)
Hadamard state: (|0> + |1>) ⊗ ... ⊗ (|0> + |1>)>
Note that both these states are very relevant in quantum computing:
The GHZ state is the unmagnetized ground state of the Ising model,
The Hadamard state is the input in many early quantum algorithms.

In [ ]:
function ghz_state(n)
    return nothing
end

function hadamard_state(n)
    return nothing
end

hadamard_state (generic function with 1 method)

In [ ]:
function test_states()
    @assert ghz_state(3) == [1, 0, 0, 0, 0, 0, 0, 1]
    @assert hadamard_state(3) == [1, 1, 1, 1, 1, 1, 1, 1]
    return nothing
end

test_states()

LoadError: AssertionError: ghz_state(3) == [1, 0, 0, 0, 0, 0, 0, 1]

hint: recursively call tensor, for example by using foldl(tensor, .)

solution:

function ghz_state(n)
    return foldl(tensor, [[1,0] for _ in 1:n]) + foldl(tensor, [[0,1] for _ in 1:n])
end

function hadamard_state(n)
    return foldl(tensor, [[1,1] for _ in 1:n])
end

## Key takeaway:  The GHZ state is very sparse in this representation, the Hadamard state is not

# Exercise 3: 5 minutes

Print the GHZ and Hadamard states for 30 qubits.
Compute the time it takes to do the inner product <hadamard_state(n), hadamard_state(n)>
for n ranging up to 25-30 (depending on the speed of your machine)

In [ ]:
function time_inner(n)
    t = @timed 1+1 #replace this with a computation of inner product
    return t[:time]
end

error("do some stuff here to print/plot the timings")

LoadError: do some stuff here to print/plot the timings

hint: use LinearAlgebra.dot

## Key takeaway:  
## The length of both the GHZ and Hadamard states increases as 2^n
## The time to compute the inner product increases exponentially in n.
## Even on very big computers, going above 40-50 qubits is impossible.

To be able to simulate large numbers of qubits, a better representation of the relevant states in the Hilbert space is needed.
For a state like the GHZ state, which is sparse in the basis above, it should be obvious that such a represenation exists.
That there is a good representation that encapsulates all interesting states (such as the Hadamard state) is not obvious.

During the morning lecture, it was explained that the Tensor Network representation seems to fit this bill, and why.
We will now redo the above exercises in the Tensor Network representation, and see the benefits it provides.

# Exercise 4: 15 minutes

Construct tensor network representations of the
GHZ state:      (|0> ⊗ ... ⊗ |0>) + (|1> ⊗ ... |1>)
Hadamard state: (|0> + |1>) ⊗ ... ⊗ (|0> + |1>)>
In ITensors such a representation is called an MPS (matrix product state)

In [ ]:
function tn_ghz_state(n)
    return nothing
end

function tn_hadamard_state(n)
    return nothing
end

tn_hadamard_state (generic function with 1 method)

In [ ]:
function test_tn_states()
    N = 5   

    function elt(ϕ, inds)
        s = siteinds(ϕ)
        V = ITensor(1.)
        for j=1:N
            V *= (ϕ[j]*state(s[j],inds[j]))
        end
        return scalar(V)
    end

    ψ = tn_ghz_state(N)
    @assert elt(ψ, [1,1,1,1,1]) ≈ 1    
    @assert elt(ψ, [2,2,2,2,2]) ≈ 1    
    @assert elt(ψ, [1,1,1,2,1]) ≈ 0
    

    ψ = tn_hadamard_state(N)
    @assert elt(ψ, [1,1,1,2,1]) ≈ 1    
    @assert elt(ψ, [1,2,1,2,2]) ≈ 1

    return nothing
end

test_tn_states (generic function with 1 method)

hint1:  You can create an MPS using the states constructed in Exercise 2, and an array of siteinds
        https://itensor.github.io/ITensors.jl/stable/examples/MPSandMPO.html

solution:

function tn_ghz_state(n)
    sites = siteinds("S=1/2", n)
    states_up = ["Up" for _ in 1:n]
    states_dn = ["Dn" for _ in 1:n]
    return MPS(sites, states_dn) + MPS(sites, states_up)
end

function tn_hadamard_state(n)
    sites = siteinds("S=1/2", n)
    state = randomMPS(sites) #this constructs a randomly initialized product wave-function in TN form
    for s in 1:n    #manually set each qubit to the [1, 1] state
        state[s][1] = 1
        state[s][2] = 1
    end
    return state
end

## Key takeaway:  The GHZ and Hadamard state have representations with low bond dimensions. This is what makes computations with them very efficient

# Exercise 5: 15 minutes

Print the GHZ and Hadamard states for 30 qubits
Compute the time it takes to do the inner product <hadamard_state(n), hadamard_state(n)>
for n ranging up to 100
Do this using the TN representation of the states

In [ ]:
error("do some stuff here to print/plot the timings")

LoadError: do some stuff here to print/plot the timings

hint:
Use ITensors.inner.
If your code takes too long, construct the GHZ state using the solution code of Exercise 4.

# Exercise 6: 15 minutes

Construct a tensor network representation of the HWA quantum circuit from the lecture for 3 qubits.
That is, the following sequence of gates (Ry, Rx, Ry, CNOTeven, CNOTodd)
where, in the 3 qubit case, CNOTeven is a CNOT gate between qubits 1 and 2, and CNOTodd is a CNOT gate between qubits 2 and 3.

In [ ]:
function hwa_layer(sites, angles::Vector{<:Number})
  return nothing
end

hwa_layer (generic function with 1 method)

In [ ]:
function test_hwa_layer()
    N = 3
    function elt(ϕ, inds)
        s = siteinds(ϕ)
        V = ITensor(1.)
        for j=1:N
            V *= (ϕ[j]*state(s[j],inds[j]))
        end
        return scalar(V)
    end
    sites = siteinds("S=1/2", N)
    state = MPS(sites, ["Up"])
    hwa = hwa_layer(sites, [pi/2, pi/2, -pi/2, pi/2, pi/2, -pi/2, pi/2, pi/2, -pi/2])
    @assert elt(out,[2,2,1]) ≈ sqrt(2)+im*sqrt(2)
    
    return nothing
end

test_hwa_layer (generic function with 1 method)

hint1: You can see how to chain elementary gates into a circuit here: https://github.com/ITensor/ITensors.jl/blob/main/examples/autodiff/ops/vqe.jl
hint2: In the above file look at layer, variational_circuit, and lines 77-79

Solution:
Ry(theta) = cos(theta/2)*[1 0; 0 1] + im*sin(theta/2)*[0 -im; im 0]
Rx(theta) = cos(theta/2)*[1 0; 0 1] + im*sin(theta/2)*[0 1; 1 0]
ITensors.op(::OpName"rotations", ::SiteType"Qubit"; t1::Number, t2::Number, t3::Number) = Ry(t3)*Rx(t2)*Ry(t1)

function hwa_layer(sites, angles::Vector{<:Number})
  layer = Prod{Op}()
  for i in 1:length(sites)
    layer = Op("rotations", i; t1=angles[1+3*(i-1)],  t2=angles[2+3*(i-1)],  t3=angles[3+3*(i-1)])*layer
  end
  layer = Op("CNOT", 1, 2)*layer
  layer = Op("CNOT", 2, 3)*layer
  return Prod{ITensor}(layer, sites)
end

## Key takeaway:  Every quantum circuit has a natural Tensor Network representation

# Exercise 7: optional

Create a randomly initialized MPS, and orthogonalize it (put it in canonical form).
Experiment with the effect on the bond dimension of the state.

In [ ]:
error("Create a random state, and orthogonalize it.")

LoadError: Create a random state, and orthogonalize it.

hint:
call orthogonalize or orthogonalize!

solution:

sites = siteinds(2, 5)
ψ = randomMPS(sites; linkdims = 5)
print(ψ)
orthogonalize!(ψ, 3)
print(ψ)
a = matrix(ψ[1], inds(ψ[1]))
print(a)
b = array(ψ[2], inds(ψ[2]))
print(b)

## key takeaway: The bond dimension of a random state can already be quite well reduced by using the canonical form, demonstrating how efficient the Tensor Network representation is.

# Exercise 8: optional

Verify that all the tensors to the left of the orthogonality center are left-orthogonal (contracting the tensor with itself along the site index, and the bond index pointing away from the center yields the identity matrix), and the tensors to the right of the orthogonality center are right-orthogonal.

In [ ]:
error("do the contractions")

LoadError: do the contractions

hint:
you can copy any tensor, and then prime some of its indexes. Contracting the tensor with its copy will contract along the unprimed indices. https://itensor.github.io/ITensors.jl/stable/ITensorType.html#Priming_and_tagging_ITensor

solution:

sites = siteinds(2, 5)
ψ = randomMPS(sites; linkdims = 5)
orthogonalize!(ψ, 3)
println(ψ)
a = ψ[1]
b = deepcopy(a)
prime!(b; tags="l=1")
c = matrix(a*b)
println(c)
d = ψ[2]
e = deepcopy(d)
prime!(e; tags="l=2")
f = matrix(d*e)
println(f)
#etc.

## Key takeaway: the orthogonality properties allow one to save computational resources, since many of the contractions occuring on measurement of local operators are guaranteed to be trivial